# 📊 Injury Impact Analysis

Analyze the impact of player injuries on team performance.

**Features:**
- Calculate player impact factors
- Compare manual vs data-driven impacts
- Analyze individual players
- Generate impact reference data

---

## 🧩 Setup

In [ ]:
import os
import sys
from pathlib import Path

print("🔍 Locating QEPC project root...\n")

# Try direct import first
try:
    from notebook_context import *
    print("✅ Imported notebook_context directly")
    
except ModuleNotFoundError:
    print("ℹ️  notebook_context not on path, searching...")
    
    # Search current directory and parents
    cwd = Path.cwd()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    
    found_root = None
    for root in candidates:
        if (root / "notebook_context.py").exists():
            found_root = root
            print(f"   Found at: {root}")
            break
    
    if found_root is None:
        raise FileNotFoundError(
            f"❌ Could not find notebook_context.py\n"
            f"   Searched: {cwd} and parent directories\n"
            f"   Ensure you're in the qepc_project folder"
        )
    
    # Add to path and re-import
    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    
    from notebook_context import *
    print("✅ Imported after path adjustment")

# Verify project_root is defined
try:
    project_root
except NameError:
    project_root = Path.cwd()
    print("⚠️  project_root not defined, using CWD")

print(f"\n📁 Project Root: {project_root}")
print(f"📂 Working Dir:  {os.getcwd()}")
print("\n" + "="*60)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

print("✅ Imports loaded")
print(f"📁 Project root: {project_root}")

---

## 📥 Load Player Statistics

In [ ]:
# Load player statistics
player_stats_path = project_root / "data" / "raw" / "PlayerStatistics.csv"

if not player_stats_path.exists():
    print("❌ PlayerStatistics.csv not found!")
    print(f"   Expected at: {player_stats_path}")
else:
    player_stats = pd.read_csv(player_stats_path)
    print(f"✅ Loaded {len(player_stats)} player records")
    print(f"\n📊 Columns available:")
    print(player_stats.columns.tolist())
    
    print(f"\n🔍 Sample data:")
    display(player_stats.head())

---

## ⚙️ Calculate Impact Factors

Impact factor represents how much a player's absence affects team performance.

**Formula:**
- Base: Usage Rate (what % of possessions the player uses)
- Adjustment: On/Off rating differential
- Scale: 0.60 (minimal impact) to 1.00 (no impact when OUT)

In [ ]:
def calculate_impact_factor(row):
    """
    Calculate injury impact factor for a player.
    
    Returns:
        float: Impact when absent (0.60-1.00)
               Lower = bigger impact when missing
    """
    # Default values if columns missing
    usage_rate = row.get('USG%', 0.20)  # Default 20% usage
    minutes = row.get('MP', 0)
    games = row.get('G', 0)
    
    # Minutes per game
    mpg = minutes / games if games > 0 else 0
    
    # Base impact on usage and minutes
    if mpg < 15:  # Bench player
        base_impact = 0.95
    elif mpg < 25:  # Role player
        base_impact = 0.85
    elif mpg < 32:  # Starter
        base_impact = 0.75
    else:  # Star player
        base_impact = 0.65
    
    # Adjust by usage rate
    if isinstance(usage_rate, str):
        usage_rate = float(usage_rate.strip('%')) / 100 if '%' in usage_rate else float(usage_rate)
    
    usage_adjustment = (usage_rate - 0.20) * 0.5  # Scale usage impact
    
    impact = base_impact - usage_adjustment
    
    # Clamp between 0.60 and 1.00
    return max(0.60, min(1.00, impact))


if 'player_stats' in dir():
    # Add impact column
    player_stats['Impact_Factor'] = player_stats.apply(calculate_impact_factor, axis=1)
    
    # Sort by impact (lowest = most important)
    impact_sorted = player_stats.sort_values('Impact_Factor')
    
    print("\n📊 Top 20 Highest Impact Players (when OUT):")
    print("="*60)
    
    cols_to_show = ['Player', 'Tm', 'Impact_Factor', 'MP', 'G']
    # Only show columns that exist
    cols_to_show = [col for col in cols_to_show if col in impact_sorted.columns]
    
    display(impact_sorted[cols_to_show].head(20))
    
    print(f"\n📈 Impact Factor Statistics:")
    print(f"   Mean: {player_stats['Impact_Factor'].mean():.3f}")
    print(f"   Median: {player_stats['Impact_Factor'].median():.3f}")
    print(f"   Min (highest impact): {player_stats['Impact_Factor'].min():.3f}")
    print(f"   Max (lowest impact): {player_stats['Impact_Factor'].max():.3f}")
else:
    print("⚠️  Player stats not loaded")

---

## 💾 Save Impact Reference Data

In [ ]:
if 'player_stats' in dir() and 'Impact_Factor' in player_stats.columns:
    # Create reference file with key columns
    reference_cols = ['Player', 'Tm', 'Impact_Factor', 'MP', 'G']
    reference_cols = [col for col in reference_cols if col in player_stats.columns]
    
    impact_reference = player_stats[reference_cols].copy()
    
    # Rename for consistency
    impact_reference.rename(columns={
        'Player': 'PlayerName',
        'Tm': 'Team',
        'Impact_Factor': 'Impact'
    }, inplace=True)
    
    # Save to data folder
    output_path = project_root / "data" / "Injury_Overrides.csv"
    impact_reference.to_csv(output_path, index=False)
    
    print(f"✅ Saved impact reference to: {output_path}")
    print(f"   Rows: {len(impact_reference)}")
    
    print("\n🔍 Sample output:")
    display(impact_reference.head(10))
else:
    print("⚠️  Impact factors not calculated yet")

---

## 🔍 Analyze Specific Players

In [ ]:
# Players to analyze (customize these!)
players_to_check = [
    "LeBron James",
    "Stephen Curry",
    "Giannis Antetokounmpo",
    "Kevin Durant",
    "Jayson Tatum",
]

print("🔍 Individual Player Analysis\n")
print("="*60)

if 'player_stats' in dir():
    for player_name in players_to_check:
        # Find player in stats
        player_data = player_stats[
            player_stats['Player'].str.contains(player_name, case=False, na=False)
        ]
        
        if player_data.empty:
            print(f"\n❌ {player_name}: Not found in data")
        else:
            row = player_data.iloc[0]
            print(f"\n📊 {row.get('Player', player_name)}")
            print(f"   Team: {row.get('Tm', 'N/A')}")
            print(f"   Games: {row.get('G', 'N/A')}")
            print(f"   Minutes: {row.get('MP', 'N/A')}")
            
            mpg = row.get('MP', 0) / row.get('G', 1) if row.get('G', 0) > 0 else 0
            print(f"   MPG: {mpg:.1f}")
            
            if 'Impact_Factor' in row:
                print(f"   🎯 Impact Factor: {row['Impact_Factor']:.3f}")
                
                # Interpret impact
                if row['Impact_Factor'] < 0.70:
                    impact_level = "🔴 CRITICAL - Star player"
                elif row['Impact_Factor'] < 0.80:
                    impact_level = "🟡 HIGH - Key starter"
                elif row['Impact_Factor'] < 0.90:
                    impact_level = "🟢 MODERATE - Role player"
                else:
                    impact_level = "⚪ LOW - Bench player"
                
                print(f"   Impact Level: {impact_level}")
else:
    print("⚠️  Player stats not loaded")

print("\n" + "="*60)

---

## 📊 Compare with Manual Overrides

In [ ]:
# Load manual injury overrides
manual_path = project_root / "data" / "Injury_Overrides.csv"

if not manual_path.exists():
    print("ℹ️  No manual injury overrides file found")
    print(f"   Expected at: {manual_path}")
elif 'impact_reference' not in dir():
    print("⚠️  Run previous cells first to calculate impact factors")
else:
    manual = pd.read_csv(manual_path)
    
    print(f"📊 Comparing Manual vs Data-Driven Impact Factors\n")
    print("="*60)
    
    # Rename manual columns for consistency
    if 'Player' in manual.columns:
        manual.rename(columns={'Player': 'PlayerName'}, inplace=True)
    if 'Tm' in manual.columns:
        manual.rename(columns={'Tm': 'Team'}, inplace=True)
    
    # Merge
    comparison = manual.merge(
        impact_reference[['PlayerName', 'Team', 'Impact']],
        on=['PlayerName', 'Team'],
        how='inner',
        suffixes=('_manual', '_data')
    )
    
    if len(comparison) == 0:
        print("⚠️  No matching players found between manual and data-driven")
    else:
        comparison['Delta'] = comparison['Impact_data'] - comparison['Impact_manual']
        
        print(f"\n📋 Comparison Results ({len(comparison)} players):\n")
        display(comparison[
            ['PlayerName', 'Team', 'Impact_manual', 'Impact_data', 'Delta']
        ].head(20))
        
        print(f"\n📈 Statistics:")
        print(f"   Mean difference: {comparison['Delta'].mean():.3f}")
        print(f"   Mean absolute difference: {comparison['Delta'].abs().mean():.3f}")
        print(f"   Max overestimate: {comparison['Delta'].max():.3f}")
        print(f"   Max underestimate: {comparison['Delta'].min():.3f}")
        
        # Players with biggest discrepancies
        print(f"\n🔍 Biggest Discrepancies:")
        biggest_diff = comparison.nlargest(5, 'Delta', keep='all')
        display(biggest_diff[['PlayerName', 'Team', 'Impact_manual', 'Impact_data', 'Delta']])

print("\n" + "="*60)

---

## 💡 Impact Factor Guide

**What does the Impact Factor mean?**

The Impact Factor represents how much a team's offensive efficiency drops when a player is OUT:

- **0.60-0.70** 🔴 CRITICAL - Superstar (30+ min, 25%+ usage)
- **0.70-0.80** 🟡 HIGH - Key starter (25-30 min, 20-25% usage)
- **0.80-0.90** 🟢 MODERATE - Role player (15-25 min)
- **0.90-1.00** ⚪ LOW - Bench player (<15 min)

**Example:**
- Team ORtg: 115.0
- LeBron James Impact: 0.70
- Adjusted ORtg (LeBron OUT): 115.0 × 0.70 = 80.5
- **Impact: -34.5 points per 100 possessions**

**Usage in QEPC:**
This impact factor is used in the lambda calculations to adjust team strength when players are injured.